In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv(r"C:\Users\Shraddha\Downloads\spaceship-titanic\train.csv")
test = pd.read_csv(r"C:\Users\Shraddha\Downloads\spaceship-titanic\test.csv")

In [3]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [5]:
print(train.shape)
print(test.shape)

(8693, 14)
(4277, 13)


In [6]:
train["source"] = "train"
test["source"] = "test"

In [7]:
data = pd.concat([train,test],axis=0)

In [8]:
train.drop('Name',axis=1,inplace=True)

In [9]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,source
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,train
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,train
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,train
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,train
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,train


In [10]:
passenger_ids = test['PassengerId']

In [11]:
data['HomePlanet'] = data['HomePlanet'].fillna(data['HomePlanet'].mode()[0])
data['CryoSleep'] = data['CryoSleep'].fillna(False)
data['Destination'] = data['Destination'].fillna(data['Destination'].mode()[0])
data['Age'] = data['Age'].fillna(data['Age'].median())
data['VIP'] = data['VIP'].fillna(False).astype(bool)


spend_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
data[spend_cols] = data[spend_cols].fillna(0)

C:\Users\Shraddha\AppData\Local\Temp\ipykernel_22960\117949469.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['CryoSleep'] = data['CryoSleep'].fillna(False)
C:\Users\Shraddha\AppData\Local\Temp\ipykernel_22960\117949469.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['VIP'] = data['VIP'].fillna(False).astype(bool)


In [12]:
data[['Deck', 'CabinNum', 'Side']] = data['Cabin'].str.split('/', expand=True)
data['CabinNum'] = pd.to_numeric(data['CabinNum'], errors='coerce').fillna(0)
data.drop(['Cabin', 'Name'], axis=1, inplace=True)

In [13]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [14]:
for col in ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']:
    data[col] = encoder.fit_transform(data[col].astype(str))

In [15]:
data['TotalSpent'] = data[spend_cols].sum(axis=1)

In [16]:
train = data[data['source'] == 'train'].drop(['source'], axis=1)
test = data[data['source'] == 'test'].drop(['source', 'Transported'], axis=1)

In [17]:
X = train.drop(['PassengerId', 'Transported'], axis=1)
y = train['Transported'].astype(bool)
X_test = test.drop(['PassengerId'], axis=1)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=20)

In [20]:
from sklearn.metrics import accuracy_score

In [21]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=6, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

C:\Users\Shraddha\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:35:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [22]:
val_preds = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_preds))

Validation Accuracy: 0.8090490797546013


In [23]:
test_preds = model.predict(X_test)

In [24]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_estimators': [300, 700, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8],
    'subsample': [0.6, 0.8, 1.0]
}

rs = RandomizedSearchCV(XGBClassifier(eval_metric='logloss'),
                        param_distributions=params, n_iter=10, cv=3, scoring='accuracy')
rs.fit(X_train, y_train)

print("Best Accuracy from CV:", rs.best_score_)
best_model = rs.best_estimator_

Best Accuracy from CV: 0.8009841032834549


In [25]:
Spaceship_Titanic= pd.DataFrame({
    'PassengerId': passenger_ids,
    'Transported': test_preds.astype(bool)  
})
Spaceship_Titanic.to_csv("Spaceship_Titanic.csv", index=False)